Install spark

In [4]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 3.6 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 11.0 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812371 sha256=cebc9651d344b3a5fa6ffa7ed3b123aa909f0fba6ef2f6c878289a03a9c090d9
  Stored in directory: /Users/walkermartin/Library/Caches/pip/wheels/11/67/ea/33c283e520b775aa7a7a0d404447e287be841a711d074d4d91
Successfully built pyspark

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
!pip show pyspark

Name: pyspark
Version: 3.5.2
Summary: Apache Spark Python API
Home-page: https://github.com/apache/spark/tree/master/python
Author: Spark Developers
Author-email: dev@spark.apache.org
License: http://www.apache.org/licenses/LICENSE-2.0
Location: /Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages
Requires: py4j
Required-by: 


In [159]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Predictive Consumption Model") \
    .getOrCreate()

24/09/06 00:28:14 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [160]:
print(spark.version)
print(spark.sparkContext.appName)
print(spark.sparkContext.master)

3.5.2
Predictive Consumption Models
local[*]


In [161]:
file_path = "Converted_Energy_Consumption.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)
df.show(df.count(), truncate=False)

+---+----+------------+-----------+------------+
|Id |Year|Fuel_Type_Id|Consumption|Units       |
+---+----+------------+-----------+------------+
|1  |1950|1           |5974.61    |Trillion BTU|
|2  |1955|1           |9006.98    |Trillion BTU|
|3  |1960|1           |12397.81   |Trillion BTU|
|4  |1965|1           |15830.08   |Trillion BTU|
|5  |1970|1           |21900.0    |Trillion BTU|
|6  |1975|1           |20241.37   |Trillion BTU|
|7  |1980|1           |20592.57   |Trillion BTU|
|8  |1985|1           |17903.12   |Trillion BTU|
|9  |1990|1           |19864.26   |Trillion BTU|
|10 |1995|1           |23006.45   |Trillion BTU|
|11 |2000|1           |24172.99   |Trillion BTU|
|12 |2005|1           |22806.5    |Trillion BTU|
|13 |2010|1           |24954.13   |Trillion BTU|
|14 |2011|1           |25358.17   |Trillion BTU|
|15 |2012|1           |26457.37   |Trillion BTU|
|16 |2013|1           |27096.58   |Trillion BTU|
|17 |2014|1           |27550.35   |Trillion BTU|
|18 |2015|1         

In [162]:
df.printSchema()
df.describe().show()

root
 |-- Id: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Fuel_Type_Id: integer (nullable = true)
 |-- Consumption: double (nullable = true)
 |-- Units: string (nullable = true)

+-------+------------------+------------------+------------------+------------------+------------+
|summary|                Id|              Year|      Fuel_Type_Id|       Consumption|       Units|
+-------+------------------+------------------+------------------+------------------+------------+
|  count|               218|               218|               218|               218|         218|
|   mean|             109.5|2003.1376146788991| 5.068807339449541|  9448.71871559633|        NULL|
| stddev|63.075351762792415|19.639595052423868|2.7153496858835537|13111.752175908192|        NULL|
|    min|                 1|              1950|                 1|              10.0|Trillion BTU|
|    max|               218|              2023|                 9|          43157.08|Trillion BTU|
+----

In [184]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, MinMaxScaler
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator



scaler = MinMaxScaler(inputCol="Year", outputCol="Year_Scaled")

rf = RandomForestRegressor(
    featuresCol="features",
    labelCol="Consumption",
    numTrees=2, 
    maxDepth=2
)

# Define Pipeline
pipeline = Pipeline(stages=[
    scaler, 
    rf
])

# Assemble features
assembler = VectorAssembler(
    inputCols=["Fuel_Type_OHE", "Year_Scaled"], 
    outputCol="features"
)

# Add assembler to the pipeline stages
pipeline_stages = [assembler] + pipeline.getStages()

# Create the pipeline
pipeline = Pipeline(stages=pipeline_stages)

# Fit the model
model = pipeline.fit(df)






IllegalArgumentException: Fuel_Type_OHE does not exist. Available: Id, Year, Fuel_Type_Id, Consumption, Units

In [179]:
# Prediction data

new_data1 = spark.createDataFrame([
    Row(Year=2025, Fuel_Type_Id=1), 
    Row(Year=2026, Fuel_Type_Id=1),
    Row(Year=2027, Fuel_Type_Id=1),
    Row(Year=2028, Fuel_Type_Id=1),
    Row(Year=2029, Fuel_Type_Id=1),
    Row(Year=2030, Fuel_Type_Id=1),
    Row(Year=2031, Fuel_Type_Id=1),
    Row(Year=2032, Fuel_Type_Id=1),
    Row(Year=2033, Fuel_Type_Id=1),
    Row(Year=2034, Fuel_Type_Id=1),
    Row(Year=2035, Fuel_Type_Id=1),
    Row(Year=2036, Fuel_Type_Id=1),
    Row(Year=2037, Fuel_Type_Id=1),
    Row(Year=2038, Fuel_Type_Id=1),
    Row(Year=2039, Fuel_Type_Id=1),
    Row(Year=2040, Fuel_Type_Id=1),
    Row(Year=2041, Fuel_Type_Id=1),
    Row(Year=2042, Fuel_Type_Id=1),
    Row(Year=2043, Fuel_Type_Id=1),
    Row(Year=2044, Fuel_Type_Id=1),
    Row(Year=2045, Fuel_Type_Id=1),
    Row(Year=2046, Fuel_Type_Id=1),
    Row(Year=2047, Fuel_Type_Id=1),
    Row(Year=2048, Fuel_Type_Id=1),
    Row(Year=2049, Fuel_Type_Id=1),
    Row(Year=2050, Fuel_Type_Id=1),
], ['Year', 'Fuel_Type_Id'])

predictions = model.transform(new_data1)
predictions.select("Year", "Fuel_Type_Id", "prediction").show()


+----+------------+-----------------+
|Year|Fuel_Type_Id|       prediction|
+----+------------+-----------------+
|2025|           1|20637.68242966433|
|2026|           1|20637.68242966433|
|2027|           1|20637.68242966433|
|2028|           1|20637.68242966433|
|2029|           1|20637.68242966433|
|2030|           1|20637.68242966433|
|2031|           1|20637.68242966433|
|2032|           1|20637.68242966433|
|2033|           1|20637.68242966433|
|2034|           1|20637.68242966433|
|2035|           1|20637.68242966433|
|2036|           1|20637.68242966433|
|2037|           1|20637.68242966433|
|2038|           1|20637.68242966433|
|2039|           1|20637.68242966433|
|2040|           1|20637.68242966433|
|2041|           1|20637.68242966433|
|2042|           1|20637.68242966433|
|2043|           1|20637.68242966433|
|2044|           1|20637.68242966433|
+----+------------+-----------------+
only showing top 20 rows



In [181]:
# Prediction data

new_data9 = spark.createDataFrame([
    Row(Year=2025, Fuel_Type_Id=9), 
    Row(Year=2026, Fuel_Type_Id=9),
    Row(Year=2027, Fuel_Type_Id=9),
    Row(Year=2028, Fuel_Type_Id=9),
    Row(Year=2029, Fuel_Type_Id=9),
    Row(Year=2030, Fuel_Type_Id=9),
    Row(Year=2031, Fuel_Type_Id=9),
    Row(Year=2032, Fuel_Type_Id=9),
    Row(Year=2033, Fuel_Type_Id=9),
    Row(Year=2034, Fuel_Type_Id=9),
    Row(Year=2035, Fuel_Type_Id=9),
    Row(Year=2036, Fuel_Type_Id=9),
    Row(Year=2037, Fuel_Type_Id=9),
    Row(Year=2038, Fuel_Type_Id=9),
    Row(Year=2039, Fuel_Type_Id=9),
    Row(Year=2040, Fuel_Type_Id=9),
    Row(Year=2041, Fuel_Type_Id=9),
    Row(Year=2042, Fuel_Type_Id=9),
    Row(Year=2043, Fuel_Type_Id=9),
    Row(Year=2044, Fuel_Type_Id=9),
    Row(Year=2045, Fuel_Type_Id=9),
    Row(Year=2046, Fuel_Type_Id=9),
    Row(Year=2047, Fuel_Type_Id=9),
    Row(Year=2048, Fuel_Type_Id=9),
    Row(Year=2049, Fuel_Type_Id=9),
    Row(Year=2050, Fuel_Type_Id=9),
], ['Year', 'Fuel_Type_Id'])

predictions = model.transform(new_data9)
predictions.select("Year", "Fuel_Type_Id", "prediction").show()

new_data_transformed = pipeline.transform(new_data9)
predictions = model.transform(new_data_transformed)




+----+------------+-----------------+
|Year|Fuel_Type_Id|       prediction|
+----+------------+-----------------+
|2025|           9|852.2707427322679|
|2026|           9|852.2707427322679|
|2027|           9|852.2707427322679|
|2028|           9|852.2707427322679|
|2029|           9|852.2707427322679|
|2030|           9|852.2707427322679|
|2031|           9|852.2707427322679|
|2032|           9|852.2707427322679|
|2033|           9|852.2707427322679|
|2034|           9|852.2707427322679|
|2035|           9|852.2707427322679|
|2036|           9|852.2707427322679|
|2037|           9|852.2707427322679|
|2038|           9|852.2707427322679|
|2039|           9|852.2707427322679|
|2040|           9|852.2707427322679|
|2041|           9|852.2707427322679|
|2042|           9|852.2707427322679|
|2043|           9|852.2707427322679|
|2044|           9|852.2707427322679|
+----+------------+-----------------+
only showing top 20 rows



AttributeError: 'Pipeline' object has no attribute 'transform'

In [175]:
print(type(model))

<class 'pyspark.ml.pipeline.PipelineModel'>


In [182]:
rf_model = model.stages[-1]
importances = rf_model.featureImportances
print("Feature Importances:", importances)

Feature Importances: (9,[0,1,2,3,4,5,6,7,8],[0.041016246283603834,0.19350432442617188,0.6318694971378909,0.030053351612061664,0.0175460250644841,0.03837812837410805,1.5203388067754965e-05,0.009482447565703062,0.03813477614790885])
